# MA2008B: Cadena de Suministro + Ruteo de Vehículos

En este notebook se implementa el modelo matemático para resolver a optimalidad el problema de cadena de suministro + ruteo de vehículos.

In [1]:
from pulp import *
import json

## Leer Datos Desde JSON

Los "-1" son None.

In [2]:
with open('input.json', 'r') as file:
    data = json.load(file)

## Modelo Matemático

In [3]:
# Inicializar PuLP
prob = LpProblem("Suministro+Ruteo", LpMinimize)

### Conjuntos del Problema

In [4]:
I = data["numProveedores"] # Cantidad de proveedores
L = data["numPoligonos"] # Cantidad de polígonos de siembra
M = data["numEspecies"] # Cantidad de especies de plantas
T = data["numPeriodos"] # Cantidad de periodos discretos
K = data["numViajes"] # Cantidad de viajes por periodo

In [5]:
# Definir los conjuntos del problema
set_I = list(range(1, I + 1))
set_L = list(range(1, L + 1))
set_M = list(range(1, M + 1))
set_T = list(range(1, T + 1))
set_K = list(range(1, K + 1))
set_V = [0] + set_L

### Parámetros del Problema

Nótese que, como en el planteamiento del modelo la mayoría de las variables se indexan empezando en 1, esta indexación debe corresponder con la de los parámetros. Como convención, se colocará, o bien una lista vacía, o bien un `None` en los espacios vacíos.

In [6]:
# Demandas de cada polígono
d = data["demandas"]

# Ofertas de cada proveedor
o = data["ofertas"]
# Costo por unidad comprada a proveedores
c = data["costosPEspecie"]

# Unidades de mano de obra necesarias por especie
g = data["manoobraPEspecie"]
# Capacidad de tratamiento del almacén por periodo
p = data["capManoobra"]

# Unidades de espacio requeridas por especie
f = data["espacioPEspecie"]
# Capaccidad total del almacén
a = data["capAlmacen"]
# Capacidad de los vehículos de envío
omega = data["capVehiculos"]

# Desfase
delta = data["desfase"]


# Tiempo de carga/descarga por unidad
rho = data["tiempoDeCarga"]
# Tiempo disponible para transporte por periodo
s = data["capTiempoPPeriodo"]

# Constante muy grande...
bigM = data["bigM"]
# Peso del costo ecnonómico en la función objetivo
e1 = data["pesoObj1"]
# Peso del costo en tiempo en la función objetivo
e2 = data["pesoObj2"]

# Costo fijo por plantar una unidad
cPlant = data["cPPlantarUnidad"]
cPedido = data["cPPedirProveedores"]

# Costos de recorrer cada arco del grafo
sigma = [[bigM, 30, 40],
         [30, bigM, 60],
         [40, 60, bigM]]

### Variales del Problema

In [7]:
# Definir las variables de decsión
x = LpVariable.dicts("x", (set_I, list(range(-delta, 1)) + set_T, set_M), lowBound = 0, cat = "Integer")
y = LpVariable.dicts("y", (set_L, set_T, set_M), lowBound = 0, cat = "Integer")
r = LpVariable.dicts("r", (set_V, set_V, set_T, set_K), cat = "Binary")
mu =  LpVariable.dicts("mu", (set_T, set_K, set_M), lowBound = 0, cat = "Integer")

# Definir las variables auxiliares
phi = LpVariable.dicts("phi", (list(range(-delta, 1)) + set_T, set_M), lowBound = 0, cat = "Integer")
w = LpVariable.dicts("w", (set_V, set_V, set_T, set_K, set_M), lowBound = 0, cat = "Integer")
q = LpVariable.dicts("q", (set_L, set_K, set_T), lowBound = 0, cat = "Integer")

h = LpVariable.dicts("h", (set_K, set_T), cat = "Binary") # Si se usa el vehículo k en t
u = LpVariable.dicts("u", (set_I, set_T), cat = "Binary") # Si se requiere al proveedor i en t

# Variables auxiliares para el objetivo
z1 = LpVariable("z1", lowBound=0, cat = "Continuous")
z2 = LpVariable("z2", lowBound=0, cat = "Continuous")

# print(phi)

### Función Objetivo

In [8]:
# La combinación lineal de los costos
prob += e1*z1 + e2*z2

Restricciones para funciones objetivo...

In [9]:
# Función objetivo sobre costos económicos
prob += z1 == lpSum(c[m][i]*x[i][t][m] for i in set_I for t in set_T for m in set_M) \
              + cPlant * lpSum(y[l][t][m] for l in set_L for t in set_T for m in set_M) \
              + cPedido * lpSum(u[i][t] for i in set_I for t in set_T)
# Función objetivo sore el tiempo invertido en el proyecto
prob += z2 == lpSum(sigma[alpha][beta]  * r[alpha][beta][t][k] for alpha in set_V for beta in set_V for t in set_T for k in set_K) \
              + 2*rho * lpSum(w[alpha][beta][t][k][m] for alpha in set_V for beta in set_V for k in set_K for t in set_T for m in set_M)

### Restricciones

#### Restricciones para la Cadena de Suministro

Restricciones de flujo para el almacén. Aquí es donde se aplica el desfase (delta).

In [10]:
for i in set_I:
    for t in set_T:
        for m in set_M:
          prob += phi[t-1-delta][m] + lpSum(x[i][t-delta][m] for i in set_I) == \
                  phi[t][m] + lpSum(y[l][t][m] for l in set_L)

# No hay plantas en almacén al inicio del proyeto
for t in range(-delta, 1):
    for m in set_M:
        prob += phi[t][m] == 0

# No se reciben pedidos de proveedores en tiempos negativos
for t in range(-delta, 1):
    for m in set_M:
        for i in set_I:
            prob += x[i][t][m] == 0

Límite en la capacidad de tratamiento por periodo de tiempo. En el modelo matemático por escrito esta restricción se había partido orignalmente en dos.

In [11]:
for t in set_T:
    prob += lpSum(g[m] * lpSum(y[l][t][m] for l in set_L) for m in set_M) <= p

Límite por la capacidad máxima del almacén.

In [12]:
for t in set_T:
    prob += lpSum(f[m] * (phi[t-1-delta][m] + lpSum(y[l][t][m] for l in set_L)) for m in set_M)  <= a

Satisfacer la demanda de los polígonos de plantación.

In [13]:
for l in set_L:
    for m in set_M:
        prob += lpSum(y[l][t][m] for t in set_T) == d[m][l]

Límite por la oferta de cada proveedor.

In [14]:
for i in set_I:
    for m in set_M:
        prob += lpSum(x[i][t][m] for t in set_T) <= o[m][i]

Encender variable si se requiere a un determinado proveedor.

In [15]:
for i in set_I:
    for t in set_T:
        prob += lpSum(x[i][t][m] for m in set_M) <= u[i][t] * bigM

#### Restricciones para el Ruteo de Vehículos

Preservar el flujo de los vehículos.

In [16]:
for t in set_T:
    for k in set_K:
        for beta in set_V:
            prob += lpSum(r[alpha][beta][t][k] for alpha in set_V) - \
                    lpSum(r[beta][gamma][t][k] for gamma in set_V) == 0

Todas las rutas visitan el almacén. Solo si ese vehículo va a repartir plantas. Si no va a repartir ninguna planta no es obligatorio que salga del almacén.

In [17]:
for t in set_T:
    for k in set_K:
        prob += lpSum(r[0][beta][t][k] for beta in set_L) == h[k][t]

# Encender o apagar la restricción anterior
for t in set_T:
    for k in set_K:
        prob += lpSum(mu[t][k][m] for m in set_M) <= h[k][t] * bigM

Nunca se supera la capacidad máxima del vehículo.

In [18]:
# Contabilizar cuantas unidades de cada especie lleva cada ruta
for t in set_T:
    for k in set_K:
        for m in set_M:
            prob += lpSum(w[alpha][beta][t][k][m] for alpha in set_V for beta in set_V) == mu[t][k][m]

# Asegurarse de que las unidades de espacio ocupadas en cada vehículo sean menores
# que su capacidad
for t in set_T:
    for k in set_K:
        prob += lpSum(f[m] * mu[t][k][m] for m in set_M) <= omega

Llevar a cada polígono exactamente las unidades asignadas

In [19]:
for t in set_T:
    for m in set_M:
        for beta in set_L:
            prob += lpSum(w[alpha][beta][t][k][m] for alpha in set_V for k in set_K) == y[beta][t][m]

Límite de tiempo para cada periodo.

In [20]:
for t in set_T:
    prob += lpSum(r[alpha][beta][t][k] * sigma[alpha][beta] for alpha in set_V for beta in set_V for k in set_K) \
            + 2*rho * lpSum(w[alpha][beta][t][k][m] for alpha in set_V for beta in set_V for k in set_K for m in set_M) <= s

Evitar subtours.

In [21]:
for t in set_T:
    for k in set_K:
        for alpha in set_L:
            for beta in set_L:
                if alpha != beta:
                    prob += q[alpha][k][t] - q[beta][k][t] + L * r[alpha][beta][t][k] <= L - 1

No existe carga transportada en arcos no utilizados.

In [22]:
for t in set_T:
    for m in set_M:
        for k in set_K:
            for alpha in set_V:
                for beta in set_V:
                    prob += w[alpha][beta][t][k][m] <= bigM * r[alpha][beta][t][k]

### Resolver

In [23]:
# Print into txt file...
import sys
sys.stdout = open("output-solution.txt", "w")

In [24]:
prob.writeLP('output-problem')
status = prob.solve()
print('Status:', LpStatus[status])

In [25]:
for var in prob.variables():
    print(f"{var.name}: {var.varValue}")